Getting familiar with all CO and PPH data

In [ ]:
import numpy as np
import geopandas as gp
import shapely as sp
import matplotlib.pyplot as plt
import contextily as cx
import cartopy as cp
from datetime import datetime as dt
from datetime import timedelta
import xarray as xr
import pandas as pd
import re
import os
from utils_filter import *
from utils_datetime import *

In [ ]:
raw_dir = '~/Downloads'
outlook_raw_location = raw_dir + '/outlooks'
report_raw_location = raw_dir + '/storm_reports'
pph_raw_location = raw_dir + '/pph'
outlook_save_location = 'data/outlooks'
report_save_location = 'data/storm_reports'
pph_save_location = 'data/pph'

# Read in Convective outlooks and filter moderate days

In [ ]:
# Read convective outlooks into outlooks
year_list = [[1987, 1991], [1992, 1999], [2000, 2007], [2008, 2015], [2016, 2023]]
for years, i in zip(year_list, range(len(year_list))):
    print('reading file ' + str(i) + ', years ' + str(years[0]) +'-' + str(years[1]))
    if i == 0:
        outlooks_original = gp.read_file(outlook_raw_location + '/outlooks_' + str(years[0]) + '01010000_' + str(years[1]) + '12312359')
    else:
        outlooks_original = outlooks_original.append(gp.read_file(outlook_raw_location + '/outlooks_' + str(years[0]) + '01010000_' + str(years[1]) + '12312359'))
print('files read')
    
outlooks_original



In [ ]:
# make dates datetime
outlooks = outlooks_original
outlooks['ISSUE'] = parse_datetime(outlooks['ISSUE'])
outlooks['EXPIRE'] = parse_datetime(outlooks['EXPIRE'])
outlooks['PRODISS'] = parse_datetime(outlooks['PRODISS'])

In [ ]:
# reset incicies
outlooks = outlooks.reset_index(drop=True)
outlooks

In [ ]:
outlooks = fix_month_issue(outlooks)    
outlooks

In [ ]:
# add column with just valid date
outlooks['DATE'] = create_dates(outlooks['EXPIRE'], -1)

# identify dates with MDT
mod_dates = identify_dates_above_threshold(outlooks, 'MDT')

In [ ]:
# Plot number of MDT days by year
years_of_mdt = get_years(mod_dates)
plt.hist(years_of_mdt, bins=range(min(years_of_mdt), max(years_of_mdt) + 1, 1))

In [ ]:
# dataframe containing only outlooks for days in which there was a MDT risk
mdt_outlooks = outlooks[outlooks['DATE'].isin(mod_dates)]

# convert datetimes back to strings
outlooks = revert_all_datetimes(outlooks)
mdt_outlooks = revert_all_datetimes(mdt_outlooks)

# save dataframes
outlooks.iloc[:int(len(outlooks)/2)].to_file(outlook_save_location + '/all_outlooks_1.shp')
outlooks.iloc[int(len(outlooks)/2)+1:].to_file(outlook_save_location + '/all_outlooks_2.shp')
mdt_outlooks.to_file(outlook_save_location + '/mdt_outlooks.shp')


# Now read, combine, filter (to mdt), and save PPH data

In [ ]:
hazard_types = ['wind', 'sig_wind', 'hail', 'sig_hail', 'tor', 'sig_tor']
for hazard, i in zip(hazard_types, range(len(hazard_types))):
    print('reading in ' + hazard + ' pph')
    if i == 0:
        pph_data = xr.open_dataset(pph_raw_location + '/pper_' + hazard + '_1979_2022.nc')
    else:
        new_data = xr.open_dataset(pph_raw_location + '/pper_' + hazard + '_1979_2022.nc')
        pph_data = xr.merge([pph_data, new_data])




In [ ]:
# select pph data on days with mdt risk
pph_data_mod = pph_data.sel(time=pph_data.time.dt.date.isin(mod_dates.tolist()))

In [ ]:
# save full and moderate pph datasets
pph_data.to_netcdf(pph_save_location + '/all_pph.nc')
pph_data_mod.to_netcdf(pph_save_location + '/mdt_pph.nc')

# Read in, combine, filter, and save raw storm reports

In [ ]:
columns =['STATE', 'EVENT_TYPE', 'CZ_TYPE', 'CZ_NAME', 'CZ_NAME', 'WFO', 'BEGIN_DATE_TIME', 'CZ_TIMEZONE', 'END_DATE_TIME', 'INJURIES_DIRECT', 'INJURIES_INDIRECT', 'DEATHS_DIRECT', 'DEATHS_INDIRECT', 'DAMAGE_PROPERTY', 'DAMAGE_CROPS', 'SOURCE', 'MAGNITUDE', 'MAGNITUDE_TYPE', 'TOR_F_SCALE', 'TOR_LENGTH', 'TOR_WIDTH', 'TOR_OTHER_WFO', 'TOR_OTHER_CZ_STATE', 'TOR_OTHER_CZ_FIPS', 'TOR_OTHER_CZ_NAME', 'BEGIN_RANGE', 'BEGIN_AZIMUTH', 'BEGIN_LOCATION', 'END_RANGE', 'END_AZIMUTH', 'END_LOCATION', 'BEGIN_LAT', 'BEGIN_LON', 'END_LAT', 'END_LON']
event_types = ['Funnel Cloud', 'Hail', 'Marine Hail', 'Marine Thunderstorm Wind', 'Thunderstorm Wind', 'Tornado', 'Waterspout']




In [ ]:
# read in reports and combine into all_reports
first = True
for file in os.listdir(os.fsencode(report_raw_location)):
    filename = os.fsdecode(file)
    if 'StormEvents_details-ftp_v1.0_d' in filename:
        print('reading' + filename)
        reports = gp.read_file(report_raw_location + '/' + filename)
        if first:
            all_reports = filter_reports(reports, columns, event_types)
            first = False
        else:
            all_reports = all_reports.append(filter_reports(reports, columns, event_types))


In [ ]:
# filter all_reports to get all_reports_mdt
all_reports['DATE'] = parse_datetime_reports(all_reports['BEGIN_DATE_TIME'])
all_reports_mdt = all_reports[all_reports['DATE'].isin(mod_dates.tolist())]



In [ ]:
# save report data
all_reports.to_csv(report_save_location + '/all_reports.csv')
all_reports_mdt.to_csv(report_save_location + '/mdt_reports.csv')
